In [1]:
import ipynb.fs.full.quick_functions as xyz

df = xyz.get_oecd_data(["MEI_FIN"])
df.head()

12:01:15
got response for MEI_FIN
12:01:38
done with MEI_FIN
12:01:43


value
SUBJECT  LOCATION FREQUENCY TIME_PERIOD          
CCRETT01 AUS      A         2000         77.11442
                            2001         74.19947
                            2002         78.13049
                            2003         88.09561
                            2004         95.33735

In [2]:
df2 = df.reset_index()
mydict = {"long_term":"IRLT",
          "short_term":"IR3TIB"}

mysubject = "long_term"
mydict[mysubject]

df_rates = df2.query(f"SUBJECT == '{mydict[mysubject]}' and FREQUENCY == 'M'").pivot(index="TIME_PERIOD",columns="LOCATION",values="value")
df_rates.index = pd.to_datetime(df_rates.index)

<IPython.core.display.Javascript object>

In [3]:
df_rates.columns

Index(['AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHL', 'CHN', 'COL',
       'CRI', 'CZE', 'DEU', 'DNK', 'EA19', 'ESP', 'EST', 'FIN', 'FRA', 'GBR',
       'GRC', 'HRV', 'HUN', 'IDN', 'IND', 'IRL', 'ISL', 'ISR', 'ITA', 'JPN',
       'KOR', 'LTU', 'LUX', 'LVA', 'MEX', 'NLD', 'NOR', 'NZL', 'POL', 'PRT',
       'ROU', 'RUS', 'SVK', 'SVN', 'SWE', 'USA', 'ZAF'],
      dtype='object', name='LOCATION')

In [13]:
#HP filter exercise

from plotly.subplots import make_subplots

myvar = "MEX"
years = 50
short_df = df_rates[myvar].iloc[-12*years:].iloc[:-1]
myseries = short_df.fillna(method="bfill")
#A value of 1600 is suggested for quarterly data.Ravn and Uhlig suggest using a value of 6.25 (1600/4**4) 
#for annual data and 129600 (1600*3**4) for monthly data...using 1600 for some reason and working like i like it
x, y = sm.tsa.filters.hpfilter(myseries,lamb=129600)

fig = px.line()
fig = make_subplots(rows=2, cols=1)
fig.add_scatter(x=x.index, y=x, mode='lines', name='cycle',row=2,col=1)
fig.add_scatter(x=y.index, y=y, mode='lines', name='trend',row=1,col=1)
fig.add_scatter(x=myseries.index,y=myseries,name='original',row=1,col=1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
#get headline CPI

import os
df_cpi = pd.read_csv("Full_PRICES_CPI.csv")
df_cpi_headline = df_cpi.query("SUBJECT=='CPALTT01'and FREQUENCY=='M' and MEASURE == 'GY'") \
    .pivot_table(index="TIME_PERIOD",columns="LOCATION",values="value",aggfunc="mean")
df_cpi_headline.head()

print(f"df_rates shape: {df_rates.shape}")
print(f"df_cpi shape: {df_cpi_headline.shape}")

joint_columns = [x for x in df_cpi_headline.columns if x in df_rates.columns]
df_real = df_rates[joint_columns].iloc[2:] - df_cpi_headline[joint_columns].values
#this iloc cuts the df by 1 to make them equal
df_real.head()

px.line(df_real["COL"])

<IPython.core.display.Javascript object>

df_rates shape: (286, 47)
df_cpi shape: (284, 50)


<IPython.core.display.Javascript object>

In [6]:
select_countries = xyz.get_countries(df_real)
xyz.create_chart_grid(df_real[select_countries],4,5,1,1,f"Real Rates (Using {mysubject} rates and Headline Inflation)")

In [7]:
df_real

LOCATION,AUT,BEL,BRA,CAN,CHE,CHL,CHN,COL,CRI,CZE,...,NLD,NOR,POL,PRT,RUS,SVK,SVN,SWE,USA,ZAF
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
2000-03-01,4.440260,3.844044,3.146887,3.842469,2.569597,NaN,NaN,NaN,NaN,NaN,...,3.679692,3.239447,NaN,3.515936,12.01537,NaN,NaN,5.061954,3.521108,11.271695
2000-04-01,3.850656,3.584340,3.137913,3.194953,2.452302,NaN,NaN,NaN,NaN,2.732496,...,3.466663,2.917586,NaN,3.691816,16.01144,NaN,NaN,4.488771,2.768116,11.939492
2000-05-01,3.720109,3.442656,4.076692,3.171522,2.707070,NaN,NaN,NaN,NaN,2.722496,...,3.562169,3.647260,NaN,4.132095,18.55826,NaN,NaN,4.307471,2.682424,11.400169
2000-06-01,3.662077,3.481153,4.225283,3.733292,2.605522,NaN,NaN,NaN,NaN,3.235879,...,3.428891,3.494372,NaN,3.457539,17.87249,NaN,NaN,4.409600,3.031408,9.784545
2000-07-01,3.854704,3.432859,3.784380,3.490253,2.395471,NaN,NaN,NaN,NaN,3.411787,...,3.221757,3.347223,NaN,3.074137,10.04400,NaN,NaN,4.301031,2.861071,8.820635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-01,-6.579148,-2.554130,3.095212,-1.067693,-1.595343,-4.601931,2.54,-2.67287,NaN,-8.360570,...,-2.424860,-2.960508,-8.79,-2.619613,NaN,-10.15833,-6.023527,-8.007000,-1.180320,4.817569
2023-07-01,-5.784231,-2.057917,3.064171,0.094528,-1.178705,-3.396323,2.46,-2.14026,NaN,-6.910950,...,-3.216279,-3.022048,-7.44,-0.754544,NaN,-8.25403,-4.956498,-7.146890,-0.147609,5.135143
2023-08-01,-4.801973,-0.934376,3.838526,0.841795,-0.760877,-1.925681,2.56,-1.42962,NaN,-5.443704,...,-2.778086,-2.556458,-5.91,-0.084032,NaN,-7.08257,-3.444833,-6.559128,1.200822,6.059151


In [8]:
myvar = "MEX"
years = 20
short_df = df_real[myvar].iloc[-12*years:].iloc[:-1]
myseries = short_df.fillna(method="bfill")
#A value of 1600 is suggested for quarterly data.Ravn and Uhlig suggest using a value of 6.25 (1600/4**4) 
#for annual data and 129600 (1600*3**4) for monthly data...using 1600 for some reason and working like i like it
x, y = sm.tsa.filters.hpfilter(myseries,lamb=1600)

fig = px.line()
fig = make_subplots(rows=2, cols=1)
fig.add_scatter(x=x.index, y=x, mode='lines', name='cycle',row=2,col=1)
fig.add_scatter(x=y.index, y=y, mode='lines', name='trend',row=1,col=1)
fig.add_scatter(x=myseries.index,y=myseries,name='original',row=1,col=1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>